In [ ]:
# 设置工作路径
import os
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr")
!ls

In [ ]:
!cdo info  -mulc,31536000 -yearmean pr_Amon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_201001-201412.nc

In [ ]:
# 合并不同时间的同模式文件
# 运行前在路径中新建“tas_mergetime”文件夹
import subprocess

os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_mergetime") # 新建文件夹
# 获取当前目录下所有文件的列表
file_list = os.listdir()
# 去除文件名后续日期
for i in range(len(file_list)):
    file_list[i]=file_list[i][0:-19]
# 去除重复项
file_list=list(set(file_list)) 

for file in file_list:
    # python文件路径变量定义
    file_path = file+'*.nc'
    output_path = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_mergetime/'+file+'mergetime.nc'
    # 构建cdo命令
    command = f'cdo -mergetime {file_path} {output_path}'
    # 运行cdo命令
    subprocess.run(command, shell=True)

In [ ]:
# 修改处理路径
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_mergetime")
!ls

In [ ]:
# 筛选进行运算的年份(时间)
# 运行前在路径中新建“tas_seltime”文件夹
import subprocess

os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_seltime") # 新建文件夹
# 获取路径文件列表
file_list = os.listdir()
# 筛选模式的数据年份
for file in file_list:
    # 定义路径变量：文件、历史文件、预测文件
    file_path = file
    # 分historical与其它模式分别构建命令
    output_path1 = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_seltime/'+file[0:-12]+'197101-200012.nc'
    output_path2 = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_seltime/'+file[0:-12]+'200101-201412.nc'
    output_path3 = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_seltime/'+file[0:-12]+'201501-206012.nc'
    # 构建cdo命令
    command1 = f'cdo -selyear,1971/2000 {file_path} {output_path1}'
    command2 = f'cdo -selyear,2001/2014 {file_path} {output_path2}'
    command3 = f'cdo -selyear,2015/2060 {file_path} {output_path3}'
    # 分类运行时间筛选的cdo命令
    if "historical" in file:
        subprocess.run(command1, shell=True)
        subprocess.run(command2, shell=True)
    else:
        subprocess.run(command3, shell=True)

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_seltime")
!ls

In [ ]:
!cdo showyear pr_Amon_NorESM2-MM_historical_r1i1p1f1_200101-201412.nc

In [ ]:
# 求各模式文件的年总降水
# 运行前在路径中新建“pe_yearsum”文件夹

os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_seltime")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_yearsum") # 新建文件夹
file_list = os.listdir()
# 筛选模式的数据年份
for file in file_list:
    # 定义路径变量：文件、历史文件、预测文件
    file_path = file
    output_path1 = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_yearsum/'+file[0:-16]+'1971-2000_yearsum.nc'
    output_path2 = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_yearsum/'+file[0:-16]+'2001-2014_yearsum.nc'
    output_path3 = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_yearsum/'+file[0:-16]+'2015-2060_yearsum.nc'
    # 构建cdo命令
    command1 = f'cdo -yearsum {file_path} {output_path1}'
    command2 = f'cdo -yearsum {file_path} {output_path2}'
    command3 = f'cdo -yearsum {file_path} {output_path3}'
    # 分类运行时间筛选的cdo命令
    if "197101-200012" in file:
        subprocess.run(command1, shell=True)
    elif "200101-201412" in file:
        subprocess.run(command2, shell=True)
    else:
        subprocess.run(command3, shell=True)

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_yearsum")
!ls

In [ ]:
!cdo info pr_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_1971-2000_yearsum.nc

In [ ]:
!cdo griddes /media/sf_VM_VirtualBox_Ubnutu/obs_pr_1971-2000.nc

In [ ]:
# 修正格网格式

# 运行前在路径中新建“tas_cngrid”文件夹
import subprocess
import os
# 定义处理路径
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_yearsum")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_cngrid") # 新建文件夹
# 提取格网形式
!cdo griddes /media/sf_VM_VirtualBox_Ubnutu/obs_pr_1971-2000.nc >grid_cn
# 修改格网形式
!sed -i "s/generic/lonlat/g" grid_cn
grid_cn=!cat grid_cn

file_list = os.listdir()

for file in file_list:
    # 定义路径变量：文件、输出文件
    file_path = file
    output_path = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_cngrid/'+file[0:-3]+'_cngrid.nc'
    # 构建cdo命令
    command = f'cdo setgrid,grid_cn {file_path} {output_path}'
    # 运行cdo命令
    subprocess.run(command, shell=True)
# 修改原观测文件格网形式至统一
!cdo setgrid,grid_cn /media/sf_VM_VirtualBox_Ubnutu/obs_pr_1971-2000.nc /media/sf_VM_VirtualBox_Ubnutu/obs_pr_1971-2000_cngrid.nc

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_cngrid")
!ls

In [ ]:
!cdo griddes /media/sf_VM_VirtualBox_Ubnutu/obs_pr_1971-2000_cngrid.nc

In [ ]:
# 裁剪研究区域

# 运行前在路径中新建“tas_china”文件夹
import os
import subprocess
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_cngrid")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_china") # 新建文件夹
# 筛选区域
# 获取文件列表
file_list = os.listdir()

for file in file_list:
    # 定义路径变量：文件、输出文件
    file_path = file
    output_path = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_china/'+file[0:-3]+'_china.nc'
    # 构建cdo命令
    command = f'cdo -sellonlatbox,67,137,14,55 {file_path} {output_path}'
    # 运行cdo命令
    subprocess.run(command, shell=True)

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_china")
!ls

In [ ]:
!cdo griddes pr_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_2001-2014_yearsum_cngrid_china.nc

In [ ]:
# 降尺度
# 运行前在路径中新建“tas_china_cngridbil”文件夹
# 把grid_cn文件粘贴至工作路径
import os
import subprocess
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_china")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_china_cngridbil") # 新建文件夹
# 获取文件列表
file_list = os.listdir()

for file in file_list:
    # 定义路径变量：文件、输出文件
    file_path = file
    output_path = '/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_china_cngridbil/'+file[0:-15]+'cngridbil_china.nc'
    # 构建cdo命令
    command = f'cdo -remapbil,grid_cn {file_path} {output_path}'
    # 运行cdo命令
    subprocess.run(command, shell=True)

In [ ]:
import os
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_china_cngridbil")
!ls

In [ ]:
!cdo griddes pr_Amon_AWI-CM-1-1-MR_ssp126_r1i1p1f1_2015-2060_yearsum_cngridbil_china.nc

In [ ]:
# 模式平均

# 运行前在路径中新建“tas_modemean”文件夹
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_china_cngridbil")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean") # 新建文件夹
!cdo -ensmean  *historical_r1i1p1f1_1971-2000*.nc /media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean/pr_historical_1971-2000_yearsum_cngridbil_china.nc
!cdo -ensmean  *historical_r1i1p1f1_2001-2014*.nc /media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean/pr_historical_2001-2014_yearsum_cngridbil_china.nc
!cdo -ensmean  *ssp126*.nc /media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean/pr_ssp126_2015-2060_yearsum_cngridbil_china.nc
!cdo -ensmean  *ssp245*.nc /media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean/pr_ssp245_2015-2060_yearsum_cngridbil_china.nc
!cdo -ensmean  *ssp585*.nc /media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean/pr_ssp585_2015-2060_yearsum_cngridbil_china.nc

In [ ]:
import os
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean")
!ls

In [ ]:
!cdo griddes pr_historical_2001-2014_yearsum_cngridbil_china.nc

In [ ]:
!cdo -timmean  pr_historical_1971-2000_yearsum_cngridbil_china.nc pr_historical_1971-2000_timmean_cngridbil_china.nc

In [ ]:
!cdo info -mulc,86400 pr_historical_1971-2000_timmean_cngridbil_china.nc

In [ ]:
!cdo info -mulc,0.1 /media/sf_VM_VirtualBox_Ubnutu/obs_pr_1971-2000_cngrid.nc

In [ ]:
# 求加性偏差
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean")
!cdo -s -f nc -sub -mulc,0.1 /media/sf_VM_VirtualBox_Ubnutu/obs_pr_1971-2000_cngrid.nc -mulc,86400 pr_historical_1971-2000_timmean_cngridbil_china.nc pr_add_delta.nc

In [ ]:
!cdo info pr_add_delta.nc

In [ ]:
# 订正gcm

# 运行前在路径中新建“tas_downscaled”文件夹
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/pr_add_downscaled") # 新建文件夹
!cdo -s -f nc -add  -mulc,86400 pr_historical_2001-2014_yearsum_cngridbil_china.nc pr_add_delta.nc /media/sf_VM_VirtualBox_Ubnutu/pr_add_downscaled/pr_historical_2001-2014_yearsum__downscaled_china.nc
!cdo -s -f nc -add  -mulc,86400 pr_ssp126_2015-2060_yearsum_cngridbil_china.nc pr_add_delta.nc /media/sf_VM_VirtualBox_Ubnutu/pr_add_downscaled/pr_ssp126_2015-2060_yearsum_downscaled_china.nc
!cdo -s -f nc -add  -mulc,86400 pr_ssp245_2015-2060_yearsum_cngridbil_china.nc pr_add_delta.nc /media/sf_VM_VirtualBox_Ubnutu/pr_add_downscaled/pr_ssp245_2015-2060_yearsum_downscaled_china.nc
!cdo -s -f nc -add  -mulc,86400 pr_ssp585_2015-2060_yearsum_cngridbil_china.nc pr_add_delta.nc /media/sf_VM_VirtualBox_Ubnutu/pr_add_downscaled/pr_ssp585_2015-2060_yearsum_downscaled_china.nc

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_add_downscaled")
!ls

In [ ]:
!cdo info pr_historical_2001-2014_yearsum__downscaled_china.nc

In [ ]:
!cdo info pr_ssp245_2015-2060_yearsum_downscaled_china.nc

In [ ]:
# 求比例偏差
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean")
!cdo -s -f nc -div -mulc,0.1 /media/sf_VM_VirtualBox_Ubnutu/obs_pr_1971-2000_cngrid.nc -mulc,86400 pr_historical_1971-2000_timmean_cngridbil_china.nc pr_mul_delta.nc

In [ ]:
!cdo info pr_mul_delta.nc

In [ ]:
# 订正gcm

# 运行前在路径中新建“tas_downscaled”文件夹
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_111/pr_modemean")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/pr_mul_downscaled") # 新建文件夹
!cdo -s -f nc -mul  -mulc,86400 pr_historical_2001-2014_yearsum_cngridbil_china.nc pr_mul_delta.nc /media/sf_VM_VirtualBox_Ubnutu/pr_mul_downscaled/pr_historical_2001-2014_yearsum_downscaled_china.nc
!cdo -s -f nc -mul  -mulc,86400 pr_ssp126_2015-2060_yearsum_cngridbil_china.nc pr_mul_delta.nc /media/sf_VM_VirtualBox_Ubnutu/pr_mul_downscaled/pr_ssp126_2015-2060_yearsum_downscaled_china.nc
!cdo -s -f nc -mul  -mulc,86400 pr_ssp245_2015-2060_yearsum_cngridbil_china.nc pr_mul_delta.nc /media/sf_VM_VirtualBox_Ubnutu/pr_mul_downscaled/pr_ssp245_2015-2060_yearsum_downscaled_china.nc
!cdo -s -f nc -mul  -mulc,86400 pr_ssp585_2015-2060_yearsum_cngridbil_china.nc pr_mul_delta.nc /media/sf_VM_VirtualBox_Ubnutu/pr_mul_downscaled/pr_ssp585_2015-2060_yearsum_downscaled_china.nc

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/pr_mul_downscaled")
!ls

In [ ]:
!cdo info pr_historical_2001-2014_yearsum_downscaled_china.nc

In [ ]:
!cdo info pr_ssp126_2015-2060_yearsum_downscaled_china.nc